In [ ]:
import pandas as pd

import os
import sys
import wave
import copy
import math
from keras.models import load_model
import tensorflow as tf
import keras


from Code.Misc.helper import *
from Code.Misc.features import *
from Code.Misc.models import audio_only_model, audio_text_model

import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

from graph_formating import *

# use this backend to display in separate Tk window
%matplotlib tk
#%matplotlib inline

# constants

TIME_SEC = 0.5
RATE = 44100 #16000                 # samples per second
CHUNK = int(RATE * TIME_SEC) # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
DEVICE = 5

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],
                                         enable=True)

In [3]:
# pyaudio class instance
p = pyaudio.PyAudio()

print("starting stram decleration")

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=int(CHUNK * 5),
    input_device_index = DEVICE
)

starting stram decleration


In [4]:
# set up model

multi_modal = True

if multi_modal:
    model = keras.models.load_model('../Misc/Saved_data/Trained_models' +
                                    '/Weights/Audio_Text/model')
else:
    model = keras.models.load_model('../Misc/Saved_data/Trained_models' +
                                    '/Weights/Audio_only/model')


cols = ['ang', 'exc', 'neu', 'sad']

df_pred_wav = pd.DataFrame([np.zeros(4)], columns=cols)
data_prev = []
graph_window = 30

In [5]:
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)
fig , ax = plt.subplots(3)

# basic formatting for the axes
ax[0].set_title('Emotion Prediction')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Confidence')

ax[0] = plot_line_graph(ax[0], df_pred_wav)

# show the plot
plt.show(block=False)

# for measuring frame rate
frame_count = 0
start_time = time.time()

In [6]:
import deepspeech as ds
from textblob import TextBlob
import scipy.signal as sps

version_dir = "F:\Capstone Project\Capstone---RTSD-System/Code\Misc\Saved_data\Trained_models\deep_speech_models/"
model_file_8 = version_dir + "0.8.2/deepspeech-0.8.2-models.pbmm"
scorer_file_8 = version_dir + "0.8.2/deepspeech-0.8.2-models.scorer"

deepspeech_model_8 = ds.Model(model_file_8)
deepspeech_model_8.enableExternalScorer(scorer_file_8)

print("Sample_rate = " + str(deepspeech_model_8.sampleRate()))

Sample_rate = 16000


In [7]:
from collections import deque
from scipy.io.wavfile import write as wav_write
import scipy.signal as sps
import re

WINDOW_SIZE = 0.5
WINDOW_N = 10
buffer = deque(maxlen=WINDOW_N)

CHUNK = int(RATE * WINDOW_SIZE)

#fill buffer
for i in range(WINDOW_N):
    buffer.append([])

# set default text and sentiment
text = ""
sentiment = 0
delay = 0
buffer_fill = 0
print("starting loop")
# while (True):
for x in range(0, 200):
    time_collect = time.time()

    # binary data
    data_new = stream.read(CHUNK)

    time_collect = time.time() - time_collect

    predict_time_start = time.time()
    #convert data to integers, make np array, then offset it by 127
    # data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)
    data_from_buffer = np.frombuffer(data_new, dtype=np.int16)

    buffer.append(data_from_buffer)
    buffer_fill += 1

    # for i in range(len(buffer)):
    #     data_prev = buffer[i]
    #     data_int = np.append(data_int, data_prev)
    data_int = []
    for sample in buffer:
        data_int += list(sample)

    data_int = np.array(data_int).astype('int16')
    # data_int = data_new
    if buffer_fill >= WINDOW_N:
        print("trigger deepspeech")
        number_of_samples = round(len(data_int) * float(16000) / RATE)
        data_16K = sps.resample(data_int, number_of_samples)

        wav_write("audio_export/______output.wav",
                  rate=16000, data=data_16K)

        stream_context = deepspeech_model_8.createStream()

        # feed audio array to model
        stream_context.feedAudioContent(data_16K.astype('int16'))

        # print output text
        text = stream_context.finishStream()

        print(text)

        tb_result = TextBlob(text)
        sentiment = np.array([tb_result.polarity])
        #sentiment = np.array([tb_result.polarity, tb_result.subjectivity]).reshape((-1, 1))
        print(sentiment)

        text = re.sub("(.{50})", "\\1\n", text, 0, re.DOTALL)

        buffer_fill = 0
    else:
        continue

    # Generate features from data
    st_features = calculate_features(data_int, RATE, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

    # reshape input from (34, 100) to (1, 100, 34)
    st_features = np.array([st_features.T])

    # predict on model
    with tf.device('/gpu:0'):
        if not multi_modal:
            wav_test_results = model.predict(st_features)
        else:
            wav_test_results = model.predict([st_features, sentiment])

    predict_time_total = time.time() - predict_time_start

    graphing_time_start = time.time()

    predicted_values = pd.DataFrame({cols[0]:wav_test_results[0][0],
                                 cols[1]:wav_test_results[0][1],
                                 cols[2]:wav_test_results[0][2],
                                 cols[3]:wav_test_results[0][3]
    }, index=[1])

    # pass previous values to filter function
    predicted_values = noise_filter(df_pred_wav.tail(1),
                                    predicted_values)

    df_pred_wav = df_pred_wav.append(predicted_values,
                                     ignore_index=True)

    df_pred_wav_view = df_pred_wav.tail(graph_window)
    df_pred_wav_view.reset_index(drop=True, inplace=True)
    ax[0].cla()
    ax[1].cla()

    ax[0] = plot_line_graph(ax[0], df_pred_wav_view)

    last_res = wav_test_results[0]
    ax[1] = plot_bar_graph (ax[1], last_res)

    ax[2] = plot_text([text, str(buffer_fill)], sentiment)

    graphing_time_total = time.time() - graphing_time_start
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:

        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)

        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        print('prediction_time = {f} seconds'.format(predict_time_total))

    data_prev = data_new


starting loop
trigger deepspeech
but where will forget us like so anyway i think the guy's
[0.]
trigger deepspeech


KeyboardInterrupt: 

In [ ]:
wav_write("audio_export/______output.wav",
          rate=16000, data=data_int)

In [ ]:
len(data_int)